In [6]:
query = """
        SELECT
          to_json_string ( STRUCT ( CAST(RANK() OVER(ORDER BY GENERATE_UUID()) AS STRING) AS id,
              STRUCT ( prdt_cd,
                store_cd ) AS
            values
              )) AS
        values
        FROM
          smart-ruler-304409.product_attribute_module.app_review_keywords
  """
path = "app_review_keywords"

In [7]:
import os
from pydatafabric.vault_utils import get_secrets
aws_info = get_secrets(mount_point="datafabric",path="mlops/access/data_loader")

In [8]:
bucket = aws_info["bucket_dev"]
ds = "20220725"
destination = f"s3a://{bucket}/dynamodb/{path}/dt={ds}/op=put/"

In [9]:
from pydatafabric.ye import get_spark

spark = get_spark()
spark.conf.set("fs.s3a.fast.upload.buffer", "disk")
spark.conf.set("fs.s3a.buffer.dir", "/tmp")
spark.conf.set("fs.s3a.access.key", aws_info["aws_access_key_id"])
spark.conf.set("fs.s3a.secret.key", aws_info["aws_secret_access_key"])

### gzip 압축 옵션 추가하여 저장

In [10]:
from pydatafabric.gcp import bq_to_df
import pyspark.sql.functions as f
import json

df = bq_to_df(query, spark_session=spark)

In [11]:
df.show(10, False)

+----------------------------------------------------------------------+
|values                                                                |
+----------------------------------------------------------------------+
|{"id":"59671","values":{"prdt_cd":"8809558842016","store_cd":"1050"}} |
|{"id":"106317","values":{"prdt_cd":"2500000039584","store_cd":"1106"}}|
|{"id":"129792","values":{"prdt_cd":"8801045522234","store_cd":"1140"}}|
|{"id":"148399","values":{"prdt_cd":"2500000308079","store_cd":"1000"}}|
|{"id":"178910","values":{"prdt_cd":"8809558842016","store_cd":"1143"}}|
|{"id":"125621","values":{"prdt_cd":"2500000305290","store_cd":"1160"}}|
|{"id":"156640","values":{"prdt_cd":"8809811825084","store_cd":"1095"}}|
|{"id":"73031","values":{"prdt_cd":"2429230000000","store_cd":"1144"}} |
|{"id":"189236","values":{"prdt_cd":"2500000271069","store_cd":"1141"}}|
|{"id":"12078","values":{"prdt_cd":"8801068398618","store_cd":"1141"}} |
+--------------------------------------------------

In [12]:
from pydatafabric.gcp import bq_insert_overwrite

bq_insert_overwrite(sql=query, destination="emart-datafabric.temp_1d.app_review_keywords_json")

destination: emart-datafabric.temp_1d.app_review_keywords_json
total_rows: 237978
slot_secs: 46.893



In [13]:
from pydatafabric.gcp import import_bigquery_ipython_magic
import_bigquery_ipython_magic()

In [14]:
%%bq

SELECT *
FROM temp_1d.app_review_keywords_json

Downloading: 100%|██████████| 237978/237978 [00:01<00:00, 215293.03rows/s]


BigQuery execution took 3 seconds.


,values
0,"{""id"":""4606"",""values"":{""prdt_cd"":""243852000000..."
1,"{""id"":""16119"",""values"":{""prdt_cd"":""88014923741..."
2,"{""id"":""24920"",""values"":{""prdt_cd"":""12641100000..."
3,"{""id"":""38546"",""values"":{""prdt_cd"":""11103900000..."
4,"{""id"":""45135"",""values"":{""prdt_cd"":""24263200000..."
...,...
237973,"{""id"":""189493"",""values"":{""prdt_cd"":""2500000161..."
237974,"{""id"":""201883"",""values"":{""prdt_cd"":""2500000279..."
237975,"{""id"":""204000"",""values"":{""prdt_cd"":""2500000077..."
237976,"{""id"":""208584"",""values"":{""prdt_cd"":""2500000170..."


In [15]:
df = bq_to_df("SELECT * FROM temp_1d.app_review_keywords_json")
df.show(10, False)

+----------------------------------------------------------------------+
|values                                                                |
+----------------------------------------------------------------------+
|{"id":"13985","values":{"prdt_cd":"2500000236648","store_cd":"1136"}} |
|{"id":"66166","values":{"prdt_cd":"8809300461465","store_cd":"1083"}} |
|{"id":"79163","values":{"prdt_cd":"2402860000000","store_cd":"1070"}} |
|{"id":"174530","values":{"prdt_cd":"1140010000000","store_cd":"1065"}}|
|{"id":"179715","values":{"prdt_cd":"2500000079597","store_cd":"1155"}}|
|{"id":"172507","values":{"prdt_cd":"2500000079597","store_cd":"1084"}}|
|{"id":"160391","values":{"prdt_cd":"2500000139574","store_cd":"1046"}}|
|{"id":"231762","values":{"prdt_cd":"2402900000000","store_cd":"1000"}}|
|{"id":"228330","values":{"prdt_cd":"2429230000000","store_cd":"1113"}}|
|{"id":"11483","values":{"prdt_cd":"2500000038549","store_cd":"1117"}} |
+--------------------------------------------------

In [16]:
df.write.mode("overwrite").option("compression", "gzip").text(destination)